In [ ]:
!pip install transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 49.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 87.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 30.2 MB/s eta 0:00:00


In [1]:
from PIL import Image
import requests
from transformers import AutoProcessor, CLIPModel

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = AutoProcessor.from_pretrained("openai/clip-vit-base-patch32")


In [3]:
import torch
from transformers import CLIPProcessor, CLIPModel, CLIPFeatureExtractor
from PIL import Image

# Load the CLIP model and processor
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = AutoProcessor.from_pretrained("openai/clip-vit-base-patch32")

In [4]:
import os

# Specify the path to the folder containing your images
folder_path = '/content/drive/MyDrive/Images'

# Initialize an empty list to store the image file paths
image_paths = []

# Iterate through the files in the folder and add image file paths to the list
for filename in os.listdir(folder_path):
    if filename.endswith(('.jpg', '.jpeg', '.png', '.gif', '.bmp')):  # Add more image file extensions as needed
        image_path = os.path.join(folder_path, filename)
        image_paths.append(image_path)

# Now, the image_paths list contains the file paths of all the images in the specified folder


In [6]:
database_embeddings = []
database_metadata = []  # This can be a list of file paths or labels

for image_path in image_paths:
    image = Image.open(image_path)
    inputs = processor(images=image, return_tensors="pt")
    image_embedding = model.get_image_features(**inputs)
    # image_embedding = model(**inputs).image_features  # Retrieve the pixel values
    database_embeddings.append(image_embedding)
    database_metadata.append(image_path)  # or any other relevant metadata


In [7]:
import pickle

# Save database_embeddings and metadata
with open('database_embeddings.pkl', 'wb') as f:
    pickle.dump({'embeddings': database_embeddings, 'metadata': database_metadata}, f)

# Load database_embeddings and metadata
with open('database_embeddings.pkl', 'rb') as f:
    saved_data = pickle.load(f)
    loaded_database_embeddings = saved_data['embeddings']
    loaded_database_metadata = saved_data['metadata']

In [25]:
def get_best_match(img_number):
  # Load and preprocess the user's query image
  # user_query_image_path = '/content/drive/MyDrive/Images/'+img_number+'.jpg'
  user_query_image_path = '/content/drive/MyDrive/shopping.jpg'

  user_query_image = Image.open(user_query_image_path)

  inputs = processor(images=user_query_image, return_tensors="pt")
  user_query_embedding = model.get_image_features(**inputs)

  # Calculate similarities between the user query and database images
  similarities = []
  # print(database_embeddings)
  for image_embedding in database_embeddings:
      similarity = torch.nn.functional.cosine_similarity(user_query_embedding, image_embedding)
      similarities.append(similarity.item())

  # print(similarities)
  # Rank and return the top matching images
  top_matches = sorted(range(len(similarities)), key=lambda i: similarities[i], reverse=True)
  print(len(top_matches))

  for idx in top_matches[:3]:
      print(f"{img_number} Image: {database_metadata[idx]}, Similarity: {similarities[idx]}")

In [26]:
get_best_match(1)

222
1 Image: /content/drive/MyDrive/Images/157.jpg, Similarity: 0.8494216203689575
1 Image: /content/drive/MyDrive/Images/97.jpg, Similarity: 0.8449047803878784
1 Image: /content/drive/MyDrive/Images/71.jpg, Similarity: 0.835048258304596


In [24]:
for i in range (200):
  get_best_match(str(i))

222
0 Image: /content/drive/MyDrive/Images/0.jpg, Similarity: 0.9999998807907104
0 Image: /content/drive/MyDrive/Images/127.jpg, Similarity: 0.8384423851966858
0 Image: /content/drive/MyDrive/Images/191.jpg, Similarity: 0.8373556733131409
222
1 Image: /content/drive/MyDrive/Images/1.jpg, Similarity: 1.0
1 Image: /content/drive/MyDrive/Images/8.jpg, Similarity: 0.858127772808075
1 Image: /content/drive/MyDrive/Images/169.jpg, Similarity: 0.8533533215522766
222
2 Image: /content/drive/MyDrive/Images/177.jpg, Similarity: 0.9999998807907104
2 Image: /content/drive/MyDrive/Images/2.jpg, Similarity: 0.9999998807907104
2 Image: /content/drive/MyDrive/Images/6.jpg, Similarity: 0.8975859880447388
222
3 Image: /content/drive/MyDrive/Images/3.jpg, Similarity: 0.9999999403953552
3 Image: /content/drive/MyDrive/Images/122.jpg, Similarity: 0.9946445226669312
3 Image: /content/drive/MyDrive/Images/61.jpg, Similarity: 0.9946445226669312
222
4 Image: /content/drive/MyDrive/Images/4.jpg, Similarity: 0.9